**Goals**

1. Set up a pipeline to incorporate the imputation
2. Do a random forest regressor to identify important features
3. Do a test run with one model (linear, most likely) that computes:
    - MSE for predicting PCIAT-Total
    - MSE for predicting sii when computed from predicted PCIAT-Total
    - MSE for predicting sii directly
    - kappa for predicting sii when computed from predicted PCIAT-Total
    - kappa for predicting sii directly
4. After getting the model working, measure these things for out-of-the box:
    - multiple linear regression
    - knn regression
    - random forest
    - support vector
    - gradient boost
    - adaboost
    - xgboost
5. After identifying a promising out-of-the-box model, try tuning it
6. Try implementing a sequential predictor (either logistic regression or random forest) that:
    - Starts by predicting 3's vs. non-threes
    - Predicts 2's vs. non-twos from the remaining cases
    - etc.
7. Try using different models for doing this sequential prediction

In [1]:
import pandas as pd
import numpy as np

from CustomImputers import *

**Loading the Data**

For the purpose of developing our model(s), we'll work with data that include the imputed outcome (PCIAT_Total and/or sii) scores AND have cleaned predictors.

In the final version of our code, we'll work with data with cleaned predictors but won't have any access to the outcome scores.

In [2]:
#Load the cleaned & outcome-imputed data
train_cleaned=pd.read_csv('train_cleaned_outcome_imputed.csv')

In [3]:
#Create an initial list of predictor and outcome columns

predictors = train_cleaned.columns.tolist()
if 'id' in predictors:
    predictors.remove('id')
if 'sii' in predictors:
    predictors.remove('sii')
predictors = [x for x in predictors if 'PCIAT' not in x]
predictors = [x for x in predictors if 'Season' not in x]

**Constructing a Random Forest for Feature Identification**

In [4]:
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import FunctionTransformer


pipe_mice = Pipeline([('mice_impute', Custom_MICE_Imputer()),
                    ('add_zones', FunctionTransformer(zone_encoder)),
                    ('rf', RandomForestRegressor(n_estimators = 300, max_features = 'sqrt', max_depth = 5, random_state = 216))])

pipe_mice.fit(train_cleaned[predictors],train_cleaned['PCIAT-PCIAT_Total'])

train_pred_mice = pipe_mice.predict(train_cleaned[predictors])

#Get feature importance from the rf inside pipe
score_mice_df = pd.DataFrame({'feature':train_cleaned[predictors].columns,
                            'importance_score': pipe_mice.named_steps['rf'].feature_importances_})

score_mice_df.sort_values('importance_score',ascending=False)


,feature,importance_score
0,Basic_Demos-Age,0.137804
4,Physical-Height,0.126698
24,PreInt_EduHx-computerinternet_hoursday,0.118666
18,BIA-BIA_FFM,0.077628
23,SDS-SDS_Total_Raw,0.074039
5,Physical-Weight,0.072494
26,ENMO_Avg_Active_Days_MVPA110,0.065296
11,FGC-FGC_CU,0.055829
19,BIA-BIA_FFMI,0.023911
13,FGC-FGC_PU,0.023766


In [4]:
keyfeatures = ['Basic_Demos-Age',
 'Physical-Height',
 'PreInt_EduHx-computerinternet_hoursday',
 'BIA-BIA_FFM',
 'SDS-SDS_Total_Raw',
 'Physical-Weight',
 'ENMO_Avg_Active_Days_MVPA110',
 'FGC-FGC_CU']

**Constructing some Linear Models**

In this section, I'll make linear models with:
* A single predictor (hours spent on the internet)
* A small number of predictors (taken from the importance scores generated above)
* All the predictors

Each of these will be run through a KFold split with a 20% validation set; for each model we'll compute several stats to compare the predictions with PCIAT scores and also with sii scores:
* MSE
* kappa

Note: Column selector documented here: https://stackoverflow.com/questions/62416223/how-to-select-only-few-columns-in-scikit-learn-column-selector-pipeline

Note: custom loss functions for linear models are documented here: https://alexmiller.phd/posts/linear-model-custom-loss-function-regularization-python/

In [ ]:
# First I'll see if I can get a pipe set up to do prediction on a split
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error


train_tt, train_ho = train_test_split(train_cleaned, test_size=0.2)

slr = Pipeline([('mice_impute', Custom_MICE_Imputer()),
                ('add_zones', FunctionTransformer(zone_encoder)),
                ('selector', ColumnTransformer([('selector', 'passthrough', ['PreInt_EduHx-computerinternet_hoursday'])], remainder="drop")),
                ('linear', LinearRegression())])

slr.fit(train_tt[predictors], train_tt['PCIAT-PCIAT_Total'])
mean_squared_error(train_ho['PCIAT-PCIAT_Total'], slr.predict(train_ho))

np.float64(334.7045879129308)

In [22]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import cohen_kappa_score
from sklearn.preprocessing import FunctionTransformer

# Start by listing the various models we'll try to identify their "out of the box" performance

models = {
'slr_pipe' : Pipeline([('mice_impute', Custom_MICE_Imputer()),
                ('add_zones', FunctionTransformer(zone_encoder)),
                ('selector', ColumnTransformer([('selector', 'passthrough', ['PreInt_EduHx-computerinternet_hoursday'])], remainder="drop")),
                ('linear', LinearRegression())]),

'mlr_key_pipe' : Pipeline([('mice_impute', Custom_MICE_Imputer()),
                ('add_zones', FunctionTransformer(zone_encoder)),
                ('selector', ColumnTransformer([('selector', 'passthrough', keyfeatures)], remainder="drop")),
                ('linear', LinearRegression())]),

'mlr_all_pipe' : Pipeline([('mice_impute', Custom_MICE_Imputer()),
                ('add_zones', FunctionTransformer(zone_encoder)),
                ('linear', LinearRegression())]),

'knn_pipe' : Pipeline([('mice_impute', Custom_MICE_Imputer()),
                ('add_zones', FunctionTransformer(zone_encoder)),
                ('knn', KNeighborsRegressor(10))]),

'svr_pipe' : Pipeline([('mice_impute', Custom_MICE_Imputer()),
                ('add_zones', FunctionTransformer(zone_encoder)),
                ('rf', SVR())]),

'rf_pipe' : Pipeline([('mice_impute', Custom_MICE_Imputer()),
                ('add_zones', FunctionTransformer(zone_encoder)),
                ('rf', RandomForestRegressor())]),

'ada_pipe' : Pipeline([('mice_impute', Custom_MICE_Imputer()),
                ('add_zones', FunctionTransformer(zone_encoder)),
                ('ada', AdaBoostRegressor())]),

'grad_pipe' : Pipeline([('mice_impute', Custom_MICE_Imputer()),
                ('add_zones', FunctionTransformer(zone_encoder)),
                ('grad', GradientBoostingRegressor())]),

'xgb_pipe' : Pipeline([('mice_impute', Custom_MICE_Imputer()),
                ('add_zones', FunctionTransformer(zone_encoder)),
                ('xgb', XGBRegressor())])
}

# Set up a list of the models and methods to organize the computation of means in the kfold split
modellist = []
for pipeline_name, pipeline_obj in models.items():
    modellist.append(pipeline_name)

methodlist = ['Predict PCIAT (MSE)', 
                'Compute SII from PCIAT (Standard Bins) (MSE)', 
                'Compute SII from PCIAT (Standard Bins) (kappa)', 
                'Compute SII from PCIAT (Modified Bins) (MSE)', 
                'Compute SII from PCIAT (Modified Bins) (kappa)',
                'Predict SII (MSE)',
                'Predict SII (rounded) (kappa)',
                'Predict SII (+0.1 rounded) (kappa)']

num_splits = 5


# Create an array with len(modellist) rows and len(methodlist) columns
output = np.zeros((len(methodlist), len(modellist), num_splits))

#Make a KFold object; setting random_state for now to help diagnose any issues with the code
kfold = KFold(num_splits, random_state=216, shuffle=True)

## i will count the split number 
i = 0
for train_index, test_index in kfold.split(train_cleaned):
    train_tt = train_cleaned.iloc[train_index]
    train_ho = train_cleaned.iloc[test_index]

    # j will enumerate the model
    j=0

    for pipeline_name, pipeline_obj in models.items():
        # print(f"Pipeline: {pipeline_name}")
        # Fit and make predictions of PCIAT
        pipeline_obj.fit(train_tt[predictors], train_tt['PCIAT-PCIAT_Total'])
        pred = pipeline_obj.predict(train_ho[predictors])
        # Compute mse for PCIAT predictions
        mse_pciat = mean_squared_error(train_ho['PCIAT-PCIAT_Total'], pred)
        #print('mse for', {pipeline_name},' for predicting PCIAT:',mse_pciat)
        # Next compute sii based on PCIAT and compute mse
        bins = [0, 30, 49,79,100]
        pred_bin = np.digitize(pred, bins)-1
        mse_sii_comp = mean_squared_error(train_ho['sii'], pred_bin)
        #print('mse for', {pipeline_name},' for predicting sii computed from PCIAT:',mse_sii_comp)
        kappa_sii_comp = cohen_kappa_score(train_ho['sii'], pred_bin, weights='quadratic')
        #print('kappa for', {pipeline_name},' for predicting sii computed from PCIAT:',kappa_sii_comp)
        # Try a slightly different set of bins
        bins_mod = [0, 25, 44, 74 ,100]
        pred_bin_mod = np.digitize(pred, bins_mod)-1
        mse_sii_comp_mod = mean_squared_error(train_ho['sii'], pred_bin_mod)
        #print('mse for', {pipeline_name},' for predicting sii computed from PCIAT with modified bins:',mse_sii_comp_mod)
        # Compute kappa for the binned predictions
        kappa_sii_comp_mod = cohen_kappa_score(train_ho['sii'], pred_bin_mod, weights='quadratic')
        #print('kappa for', {pipeline_name},' for predicting sii computed from PCIAT with modified bins:',kappa_sii_comp_mod)
        #print(f"Pipeline {pipeline_name} predictions: {y_pred}")
        output[0,j,i] = mse_pciat
        output[1,j,i] = mse_sii_comp
        output[2,j,i] = kappa_sii_comp
        output[3,j,i] = mse_sii_comp_mod
        output[4,j,i] = kappa_sii_comp_mod
        j=j+1

    j=0
    for pipeline_name, pipeline_obj in models.items():
        # Fit and make predictions of sii
        pipeline_obj.fit(train_tt[predictors], train_tt['sii'])
        pred = pipeline_obj.predict(train_ho[predictors])
        # Try two different ways of rounding the predictions
        pred_round = np.round(pred)
        pred_roundmod = np.round(pred+0.1)
        # Compute mse and kappas
        mse_sii = mean_squared_error(train_ho['sii'], pred)
        kappa_sii_round = cohen_kappa_score(train_ho['sii'], pred_round, weights='quadratic')
        kappa_sii_roundmod = cohen_kappa_score(train_ho['sii'], pred_roundmod, weights='quadratic')
        #print('mse for', {pipeline_name},' for predicting sii:',mse_sii)
        #print('kappa for', {pipeline_name},' for predicting sii with regular rounding:',kappa_sii_round)
        #print('kappa for', {pipeline_name},' for predicting sii with adjusted rounding:',kappa_sii_roundmod)
        output[5,j,i] = mse_sii
        output[6,j,i] = kappa_sii_round
        output[7,j,i] = kappa_sii_roundmod
        j=j+1
    i=i+1

# Create a new array by computing the average of the values in output along the third axis
output_avg = np.mean(output, axis=2)

In [23]:
# create a data frame from output using modellist as the names of the columns and methodlist as the names of the rows
output_df = pd.DataFrame(output_avg, columns=modellist, index=methodlist)

output_df

,slr_pipe,mlr_key_pipe,mlr_all_pipe,knn_pipe,svr_pipe,rf_pipe,ada_pipe,grad_pipe,xgb_pipe
Predict PCIAT (MSE),357.761911,304.317231,303.210991,364.319223,367.520965,309.831142,330.664890,303.065715,361.656975
Compute SII from PCIAT (Standard Bins) (MSE),0.671371,0.599800,0.593414,0.685008,0.772035,0.586578,0.596156,0.595690,0.672720
Compute SII from PCIAT (Standard Bins) (kappa),0.252972,0.377227,0.381598,0.276931,0.182760,0.382803,0.339858,0.376473,0.340306
Compute SII from PCIAT (Modified Bins) (MSE),0.623972,0.587489,0.593873,0.688216,0.634886,0.609823,0.655407,0.591137,0.709187
Compute SII from PCIAT (Modified Bins) (kappa),0.279800,0.420170,0.419586,0.318950,0.266209,0.401251,0.336452,0.420993,0.351154
Predict SII (MSE),0.540118,0.472936,0.469641,0.558287,0.542637,0.489432,0.553796,0.475840,0.547095
Predict SII (rounded) (kappa),0.279220,0.368460,0.376743,0.275728,0.297620,0.332263,0.070702,0.373602,0.359288
Predict SII (+0.1 rounded) (kappa),0.276686,0.347903,0.350055,0.276507,0.307987,0.319398,0.086913,0.367315,0.360745


**Sequential Binary Classification**

It looks like our attempts so far have under-predicted sii values of 2 and 3. I'm going to try to implement a method that first predicts whether or not the sii value is 3, then on the remaining values predict whether or not they are 2, etc.

I came up with this idea myself, but I wasn't the first one to do it. It was described on Medium: https://towardsdatascience.com/simple-trick-to-train-an-ordinal-regression-with-any-classifier-6911183d2a3c from an article by Frank and Hal

Also described on stackoverflow: https://stackoverflow.com/questions/57561189/multi-class-multi-label-ordinal-classification-with-sklearn

Some discussion of the proposed code that highlights some of its issues is on stackoverflow: https://stackoverflow.com/questions/66486947/how-to-use-ordinal-classifier

In [ ]:
import numpy as np
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.metrics import accuracy_score

class OrdinalClassifier(BaseEstimator, ClassifierMixin):

    def __init__(self, clf):
        self.clf = clf
        self.clfs = {}
        self.unique_class = np.NaN

    def fit(self, X, y):
        self.unique_class = np.sort(np.unique(y))
        if self.unique_class.shape[0] > 2:
            for i in range(self.unique_class.shape[0]-1):
                # for each k - 1 ordinal value we fit a binary classification problem
                binary_y = (y > self.unique_class[i]).astype(np.uint8)
                clf = clone(self.clf)
                clf.fit(X, binary_y)
                self.clfs[i] = clf

    def predict_proba(self, X):
        clfs_predict = {i: self.clfs[i].predict_proba(X) for i in self.clfs}
        predicted = []
        k = len(self.unique_class) - 1
        for i, y in enumerate(self.unique_class):
            if i == 0:
                # V1 = 1 - Pr(y > V1)
                predicted.append(1 - clfs_predict[0][:,1])
            elif i < k:
                # Vi = Pr(y <= Vi) * Pr(y > Vi-1)
                 predicted.append((1 - clfs_predict[i][:,1]) * clfs_predict[i-1][:,1])
            else:
                # Vk = Pr(y > Vk-1)
                predicted.append(clfs_predict[k-1][:,1])
        return np.vstack(predicted).T

    def predict(self, X):
        return self.unique_class[np.argmax(self.predict_proba(X), axis=1)]

    def score(self, X, y, sample_weight=None):
        return accuracy_score(y, self.predict(X), sample_weight=sample_weight)

**Using Neural Regression to Predict Ordinal Outcomes**

It looks like we could also use "neural regression" (PyTorch) to do the prediction. 

Here is a website that describes how to accomplish this: https://visualstudiomagazine.com/articles/2021/10/04/ordinal-classification-pytorch.aspx

The code below is copied from the website

In [ ]:
# house_price_ord.py
# predict ordinal price from AC, sq ft, style, nearest school
# PyTorch 1.8.0-CPU Anaconda3-2020.02  Python 3.7.6
# Windows 10 

import numpy as np
import torch as T
device = T.device("cpu")  # apply to Tensor or Module

# -----------------------------------------------------------

class HouseDataset(T.utils.data.Dataset):
  # AC  sq ft   style  price   school
  # -1  0.2500  0 1 0    3     0 1 0
  #  1  0.1275  1 0 0    2     0 0 1
  # air condition: -1 = no, +1 = yes
  # style: art_deco, bungalow, colonial
  # price: k=4: 0 = low, 1 = medium, 2 = high, 3 = very high
  # school: johnson, kennedy, lincoln

  def __init__(self, src_file, k):
    # k for programmtic approach
    all_xy = np.loadtxt(src_file, 
      usecols=[0,1,2,3,4,5,6,7,8], delimiter="\t",
      comments="#", skiprows=0, dtype=np.float32)

    tmp_x = all_xy[:,[0,1,2,3,4,6,7,8]]
    tmp_y = all_xy[:,5]    # 1D -- 2D will be required

    n = len(tmp_y)
    for i in range(n):  # hard-coded is easy to understand
      if int(tmp_y[i])   == 0: tmp_y[i] = 0.125
      elif int(tmp_y[i]) == 1: tmp_y[i] = 0.375
      elif int(tmp_y[i]) == 2: tmp_y[i] = 0.625
      elif int(tmp_y[i]) == 3: tmp_y[i] = 0.875
      else: print("Fatal logic error ")

    tmp_y = np.reshape(tmp_y, (-1,1))  # 2D    

    self.x_data = T.tensor(tmp_x, \
      dtype=T.float32).to(device)
    self.y_data = T.tensor(tmp_y, \
      dtype=T.float32).to(device)

  def __len__(self):
    return len(self.x_data)

  def __getitem__(self, idx):
    preds = self.x_data[idx,:]  # or just [idx]
    price = self.y_data[idx,:] 
    return (preds, price)       # tuple of two matrices 

# -----------------------------------------------------------

class Net(T.nn.Module):
  def __init__(self):
    super(Net, self).__init__()
    self.hid1 = T.nn.Linear(8, 10)  # 8-(10-10)-1
    self.hid2 = T.nn.Linear(10, 10)
    self.oupt = T.nn.Linear(10, 1)  # [0.0 to 1.0]

    T.nn.init.xavier_uniform_(self.hid1.weight)
    T.nn.init.zeros_(self.hid1.bias)
    T.nn.init.xavier_uniform_(self.hid2.weight)
    T.nn.init.zeros_(self.hid2.bias)
    T.nn.init.xavier_uniform_(self.oupt.weight)
    T.nn.init.zeros_(self.oupt.bias)

  def forward(self, x):
    z = T.tanh(self.hid1(x))
    z = T.tanh(self.hid2(z))
    z = T.sigmoid(self.oupt(z))  # 
    return z

# -----------------------------------------------------------

def accuracy(model, ds, k):
  n_correct = 0; n_wrong = 0
  acc_delta = (1.0 / k) / 2   # if k=4 delta = 0.125
  for i in range(len(ds)):    # each input
    (X, y) = ds[i]            # (predictors, target)
    with T.no_grad():         # y target is like 0.375
      oupt = model(X)         # oupt is in [0.0, 1.0]

    if T.abs(oupt - y) <= acc_delta:
      n_correct += 1
    else:
      n_wrong += 1

  acc = (n_correct * 1.0) / (n_correct + n_wrong)
  return acc

# -----------------------------------------------------------

def accuracy_old(model, ds, k):
  model.eval()
  n_correct = 0; n_wrong = 0
  for i in range(len(ds)):
    (X, Y) = ds[i]            # (predictors, target)
    with T.no_grad():
      oupt = model(X)         # computed is in 0.0 to 1.0
    if oupt >= 0.0 and oupt < 0.25 and Y == 0.125:  # ugly
      n_correct += 1
    elif oupt >= 0.25 and oupt < 0.50 and Y == 0.375:
      n_correct += 1
    elif oupt >= 0.50 and oupt < 0.75 and Y == 0.625:
      n_correct += 1
    elif oupt >= 0.75 and Y == 0.875:
      n_correct += 1
    else:
      n_wrong += 1
  acc = (n_correct * 1.0) / (n_correct + n_wrong)
  return acc

# -----------------------------------------------------------

def train(net, ds, bs, lr, me, le):
  # network, dataset, batch_size, learn_rate, 
  # max_epochs, log_every
  train_ldr = T.utils.data.DataLoader(ds,
    batch_size=bs, shuffle=True)
  loss_func = T.nn.MSELoss()
  opt = T.optim.Adam(net.parameters(), lr=lr)

  for epoch in range(0, me):
    # T.manual_seed(1+epoch)  # recovery reproducibility
    epoch_loss = 0  # for one full epoch

    for (b_idx, batch) in enumerate(train_ldr):
      (X, y) = batch           # (predictors, targets)
      opt.zero_grad()          # prepare gradients
      oupt = net(X)            # predicted prices

      loss_val = loss_func(oupt, y)  # a tensor
      epoch_loss += loss_val.item()  # accumulate
      loss_val.backward()  # compute gradients
      opt.step()           # update weights

    if epoch % le == 0:
      print("epoch = %4d   loss = %0.4f" % \
       (epoch, epoch_loss))
      # TODO: save checkpoint

# -----------------------------------------------------------

def float_oupt_to_class(oupt, k):
  end_pts = np.zeros(k+1, dtype=np.float32) 
  delta = 1.0 / k
  for i in range(k):
    end_pts[i] = i * delta
  end_pts[k] = 1.0
  # if k=4, [0.0, 0.25, 0.50, 0.75, 1.0] 

  for i in range(k):
    if oupt >= end_pts[i] and oupt <= end_pts[i+1]:
      return i
  return -1  # fatal error 

# -----------------------------------------------------------

def main():
  # 0. get started
  print("\nBegin predict House ordinal price \n")
  T.manual_seed(1)  # representative results 
  np.random.seed(1)
  
  # 1. create Dataset objects
  print("Creating Houses Dataset objects ")
  print("Converting ordinal labels to float targets ")
  train_file = ".\\Data\\houses_train_ord.txt"
  train_ds = HouseDataset(train_file, k=4)  # 200 rows

  test_file = ".\\Data\\houses_test_ord.txt"
  test_ds = HouseDataset(test_file, k=4)  # 40 rows

  # 2. create network
  print("\nCreating 8-10-10-1 neural network ")
  net = Net().to(device)
  net.train()   # set mode

  # 3. train model
  bat_size = 10
  lrn_rate = 0.010
  max_epochs = 500
  log_every = 100

  print("\nbat_size = %3d " % bat_size)
  print("lrn_rate = %0.3f " % lrn_rate)
  print("loss = MSELoss ")
  print("optimizer = Adam ")
  print("max_epochs = %3d " % max_epochs)

  print("\nStarting training ")
  train(net, train_ds, bat_size, lrn_rate, 
    max_epochs, log_every)
  print("Training complete ")

  # 4. evaluate model accuracy
  print("\nComputing model accuracy")
  net.eval()  # set mode
  acc_train = accuracy(net, train_ds, k=4) 
  print("Accuracy on train data = %0.4f" % \
    acc_train)

  acc_test = accuracy(net, test_ds, k=4) 
  print("Accuracy on test data  = %0.4f" % \
    acc_test)

  # 5. save trained model (TODO)
  print("\nSaving trained model as houses_model.h5 ")
  # model.save_weights(".\\Models\\houses_model_wts.h5")
  # model.save(".\\Models\\houses_model.h5")

  # 6. make a prediction
  print("\nPredicting house price for AC=no, sqft=2300, ")
  print(" style=colonial, school=kennedy: ")
  unk = np.array([[-1, 0.2300,  0,0,1,  0,1,0]],
    dtype=np.float32)
  unk = T.tensor(unk, dtype=T.float32).to(device) 

  with T.no_grad():
    pred_price = net(unk)
  pred_price = pred_price.item()  # scalar 0.0 to 1.0
  print("\nPredicted price raw output: %0.4f" % \
    pred_price)

  labels = ["low", "medium", "high", "very high"]
  c = float_oupt_to_class(pred_price, k=4)
  print("Predicted price ordinal label: %d " % c)
  print("Predicted price friendly class: %s " % \
    labels[c])

  print("\nEnd House ordinal price demo")

if __name__ == "__main__":
  main()

# ===========
# houses_train_ord.txt
# AC (-1 = no), sq_ft, style (one-hot)
# price (0=low, 1=med, 2=high, 3=v. high), 
# school (one-hot)
#   -1   0.1275   0   1   0   0   0   0   1
#    1   0.1100   1   0   0   0   1   0   0
#   -1   0.1375   0   0   1   0   0   1   0
#    1   0.1975   0   1   0   2   0   0   1
#   -1   0.1200   0   0   1   0   1   0   0
#   -1   0.2500   0   1   0   2   0   1   0
#    1   0.1275   1   0   0   1   0   0   1
#   -1   0.1750   0   0   1   1   0   0   1
#   -1   0.2500   0   1   0   2   0   0   1
#    1   0.1800   0   1   0   1   1   0   0
#    1   0.0975   1   0   0   0   0   0   1
#   -1   0.1100   0   1   0   0   0   1   0
#    1   0.1975   0   0   1   1   0   0   1
#   -1   0.3175   1   0   0   3   0   1   0
#   -1   0.1700   0   1   0   1   1   0   0
#    1   0.1650   0   1   0   1   0   1   0
#   -1   0.2250   0   1   0   2   0   1   0
#   -1   0.2125   0   1   0   2   0   1   0
#    1   0.1675   0   1   0   1   0   1   0
#    1   0.1550   1   0   0   1   0   1   0
#   -1   0.1375   0   0   1   0   1   0   0
#   -1   0.2425   0   1   0   2   1   0   0
#    1   0.3200   0   0   1   3   0   1   0
#   -1   0.3075   1   0   0   3   0   1   0
#   -1   0.2700   1   0   0   2   0   0   1
#    1   0.1700   0   1   0   1   0   0   1
#   -1   0.1475   1   0   0   1   1   0   0
#   -1   0.2500   0   1   0   2   0   0   1
#   -1   0.2750   1   0   0   2   0   0   1
#   -1   0.2000   1   0   0   2   1   0   0
#   -1   0.1100   0   0   1   0   1   0   0
#   -1   0.3400   1   0   0   3   0   1   0
#    1   0.3000   0   0   1   3   1   0   0
#    1   0.1550   0   1   0   1   0   1   0
#   -1   0.2150   0   1   0   1   0   0   1
#   -1   0.2900   0   0   1   3   0   1   0
#    1   0.2750   0   0   1   2   0   1   0
#    1   0.2175   0   1   0   2   0   1   0
#    1   0.2150   0   1   0   2   0   0   1
#    1   0.1050   1   0   0   1   1   0   0
#   -1   0.2775   1   0   0   2   0   0   1
#   -1   0.3225   1   0   0   3   0   1   0
#    1   0.2075   0   1   0   2   1   0   0
#   -1   0.3225   1   0   0   3   0   0   1
#    1   0.2800   0   0   1   3   0   0   1
#   -1   0.1575   0   1   0   1   0   0   1
#    1   0.3250   0   0   1   3   0   0   1
#   -1   0.2750   1   0   0   2   0   0   1
#    1   0.1250   1   0   0   1   1   0   0
#   -1   0.2325   0   1   0   2   0   0   1
#    1   0.1825   1   0   0   2   1   0   0
#   -1   0.2600   0   1   0   2   0   1   0
#   -1   0.3075   1   0   0   3   0   0   1
#   -1   0.2875   1   0   0   3   0   0   1
#    1   0.2300   0   1   0   2   0   1   0
#    1   0.3100   0   0   1   3   1   0   0
#   -1   0.2750   1   0   0   2   0   0   1
#    1   0.1125   0   1   0   0   0   0   1
#    1   0.2525   1   0   0   2   1   0   0
#    1   0.1625   0   1   0   1   0   1   0
#    1   0.1075   1   0   0   1   0   0   1
#   -1   0.2200   0   1   0   2   0   1   0
#   -1   0.2300   0   1   0   2   0   1   0
#   -1   0.3100   1   0   0   3   0   1   0
#   -1   0.2875   1   0   0   3   0   1   0
#    1   0.3375   0   0   1   3   0   0   1
#   -1   0.1450   0   0   1   0   1   0   0
#   -1   0.2650   1   0   0   2   1   0   0
#    1   0.2225   0   1   0   2   1   0   0
#   -1   0.2300   0   1   0   2   0   1   0
#    1   0.1025   0   1   0   0   0   1   0
#    1   0.1925   0   1   0   2   1   0   0
#   -1   0.2525   0   1   0   2   0   1   0
#   -1   0.1650   0   1   0   1   0   1   0
#    1   0.1650   0   1   0   1   0   1   0
#   -1   0.1300   1   0   0   1   0   1   0
#   -1   0.2900   1   0   0   3   1   0   0
#   -1   0.2175   0   1   0   1   0   0   1
#    1   0.2300   1   0   0   2   1   0   0
#   -1   0.3000   1   0   0   3   1   0   0
#    1   0.2125   0   1   0   1   1   0   0
#    1   0.2825   0   0   1   2   0   0   1
#    1   0.3125   0   0   1   3   0   1   0
#    1   0.2500   0   1   0   2   1   0   0
#   -1   0.2375   0   1   0   2   0   0   1
#    1   0.3375   0   0   1   3   0   1   0
#    1   0.2000   0   1   0   2   0   0   1
#   -1   0.2100   0   1   0   1   0   1   0
#   -1   0.3225   1   0   0   3   1   0   0
#    1   0.2375   0   0   1   2   1   0   0
#   -1   0.2250   0   1   0   2   0   1   0
#    1   0.1250   1   0   0   1   0   0   1
#   -1   0.1925   1   0   0   1   1   0   0
#   -1   0.2750   0   1   0   2   0   0   1
#    1   0.2200   0   1   0   2   1   0   0
#   -1   0.1675   0   1   0   1   1   0   0
#   -1   0.1700   0   1   0   1   0   0   1
#   -1   0.1350   0   0   1   0   0   1   0
#   -1   0.1600   0   1   0   1   0   1   0
#   -1   0.2125   0   1   0   1   0   0   1
#    1   0.1200   1   0   0   1   0   0   1
#   -1   0.2100   0   1   0   2   0   1   0
#   -1   0.1250   0   0   1   0   0   0   1
#   -1   0.2550   0   1   0   2   0   1   0
#    1   0.2750   0   0   1   2   0   1   0
#   -1   0.2200   0   0   1   1   1   0   0
#    1   0.0925   1   0   0   1   1   0   0
#    1   0.3350   0   0   1   3   0   1   0
#   -1   0.2250   0   1   0   2   0   0   1
#   -1   0.2425   0   1   0   2   1   0   0
#    1   0.1275   0   1   0   1   0   1   0
#    1   0.3350   0   1   0   3   1   0   0
#   -1   0.1850   0   1   0   1   0   0   1
#    1   0.1600   0   1   0   1   1   0   0
#   -1   0.2400   0   1   0   2   1   0   0
#    1   0.3300   0   0   1   3   0   0   1
#   -1   0.3075   1   0   0   3   1   0   0
#    1   0.2900   0   1   0   3   0   0   1
#   -1   0.0950   0   0   1   0   1   0   0
#   -1   0.1900   0   1   0   1   0   0   1
#    1   0.1375   0   1   0   1   1   0   0
#   -1   0.2100   0   1   0   1   1   0   0
#   -1   0.3025   1   0   0   3   1   0   0
#    1   0.1375   1   0   0   0   0   0   1
#   -1   0.1475   1   0   0   1   0   1   0
#    1   0.2150   0   1   0   2   1   0   0
#   -1   0.2400   0   1   0   2   1   0   0
#   -1   0.1375   0   0   1   0   0   0   1
#    1   0.2200   1   0   0   2   1   0   0
#   -1   0.1150   0   0   1   0   0   1   0
#    1   0.1825   0   0   1   2   0   1   0
#   -1   0.3225   1   0   0   3   0   0   1
#   -1   0.1450   0   0   1   0   0   0   1
#    1   0.1675   0   1   0   1   1   0   0
#    1   0.3325   0   0   1   3   0   1   0
#    1   0.1075   1   0   0   0   0   0   1
#   -1   0.1350   0   0   1   0   1   0   0
#   -1   0.1450   0   0   1   0   1   0   0
#    1   0.1575   0   1   0   1   1   0   0
#   -1   0.1825   0   1   0   1   0   0   1
#   -1   0.2450   0   1   0   2   0   1   0
#    1   0.1425   1   0   0   1   1   0   0
#    1   0.2175   0   1   0   2   0   0   1
#    1   0.2325   0   1   0   2   0   1   0
#   -1   0.2875   1   0   0   3   1   0   0
#    1   0.2625   0   1   0   2   0   0   1
#    1   0.1575   0   1   0   1   0   0   1
#    1   0.2750   0   0   1   2   1   0   0
#   -1   0.2500   0   1   0   2   1   0   0
#   -1   0.2400   0   1   0   2   0   1   0
#    1   0.1100   1   0   0   0   0   0   1
#   -1   0.2975   1   0   0   3   0   0   1
#   -1   0.1725   0   0   1   1   1   0   0
#    1   0.3225   0   0   1   3   1   0   0
#   -1   0.1450   0   0   1   0   0   0   1
#    1   0.1725   0   1   0   1   0   1   0
#    1   0.3050   0   0   1   3   1   0   0
#   -1   0.3200   1   0   0   3   0   0   1
#    1   0.1450   1   0   0   1   1   0   0
#   -1   0.3175   1   0   0   3   0   1   0
#    1   0.1475   1   0   0   1   0   1   0
#    1   0.2575   0   1   0   2   1   0   0
#    1   0.1200   1   0   0   1   0   0   1
#   -1   0.2425   0   1   0   2   0   1   0
#   -1   0.0900   1   0   0   0   1   0   0
#   -1   0.0925   0   0   1   0   1   0   0
#   -1   0.1650   0   0   1   1   0   1   0
#    1   0.1025   1   0   0   0   0   0   1
#   -1   0.1475   0   0   1   0   0   0   1
#    1   0.2225   1   0   0   2   0   0   1
#    1   0.3250   1   0   0   3   0   0   1
#    1   0.2800   0   0   1   2   1   0   0
#    1   0.2625   0   1   0   2   0   0   1
#    1   0.1450   1   0   0   1   0   1   0
#    1   0.2350   0   1   0   2   0   1   0
#   -1   0.3425   0   0   1   3   1   0   0
#   -1   0.1575   0   1   0   1   0   0   1
#   -1   0.3075   0   0   1   2   0   1   0
#   -1   0.0950   0   0   1   0   0   1   0
#   -1   0.1925   0   1   0   1   0   0   1
#    1   0.1300   1   0   0   1   1   0   0
#   -1   0.3075   1   0   0   3   0   1   0
#   -1   0.2000   0   1   0   1   1   0   0
#    1   0.2475   0   1   0   3   1   0   0
#   -1   0.2825   1   0   0   3   1   0   0
#    1   0.2425   0   1   0   3   0   1   0
#   -1   0.2625   0   0   1   2   1   0   0
#    1   0.0900   1   0   0   0   1   0   0
#    1   0.2800   0   0   1   2   0   0   1
#    1   0.2600   0   1   0   2   0   1   0
#    1   0.0900   0   1   0   0   0   1   0
#    1   0.2900   0   0   1   3   1   0   0
#    1   0.1950   0   1   0   2   0   1   0
#    1   0.2325   0   1   0   2   1   0   0
#    1   0.2025   0   1   0   1   0   1   0
#    1   0.3025   0   0   1   3   1   0   0
#   -1   0.1800   0   0   1   1   0   1   0
#   -1   0.2225   0   1   0   2   1   0   0
#   -1   0.1425   0   0   1   0   1   0   0
#   -1   0.2725   1   0   0   2   0   0   1
#  
# houses_test_ord.txt                          
#    1   0.2550   0   1   0   2   1   0   0
#    1   0.1625   0   1   0   1   0   1   0
#   -1   0.2750   1   0   0   2   1   0   0
#   -1   0.1275   0   0   1   0   0   0   1
#   -1   0.1650   0   0   1   1   0   0   1
#    1   0.1450   1   0   0   1   0   1   0
#   -1   0.3275   1   0   0   3   1   0   0
#    1   0.2175   0   1   0   2   0   1   0
#    1   0.2725   0   0   1   2   0   1   0
#   -1   0.3075   1   0   0   3   0   1   0
#   -1   0.2600   1   0   0   2   0   1   0
#   -1   0.1525   0   0   1   0   0   1   0
#   -1   0.1450   0   0   1   0   1   0   0
#    1   0.2375   0   1   0   2   0   0   1
#   -1   0.1950   0   1   0   1   0   1   0
#   -1   0.2375   0   1   0   2   0   0   1
#    1   0.2475   0   1   0   2   1   0   0
#    1   0.3150   0   0   1   3   0   0   1
#    1   0.1525   1   0   0   1   1   0   0
#    1   0.3050   0   0   1   3   0   0   1
#    1   0.2350   0   1   0   2   0   0   1
#   -1   0.1525   0   0   1   0   0   0   1
#    1   0.2550   0   1   0   2   0   0   1
#    1   0.1200   0   1   0   1   1   0   0
#    1   0.2450   0   1   0   2   1   0   0
#   -1   0.3300   1   0   0   3   0   0   1
#    1   0.3275   1   0   0   3   1   0   0
#    1   0.2300   1   0   0   2   0   1   0
#    1   0.2275   0   1   0   2   0   0   1
#    1   0.2350   1   0   0   2   1   0   0
#    1   0.1475   1   0   0   1   1   0   0
#    1   0.2850   0   0   1   3   0   0   1
#    1   0.1000   0   0   1   0   1   0   0
#    1   0.1750   0   1   0   1   1   0   0
#    1   0.3075   0   0   1   3   0   0   1
#    1   0.1550   0   1   0   1   0   0   1
#   -1   0.0925   0   0   1   0   1   0   0
#   -1   0.1300   0   0   1   0   0   0   1
#    1   0.1425   0   0   1   1   1   0   0
#    1   0.2975   0   0   1   3   0   0   1
